# 普通列和行index的相互转化
对于DataFrame来说，普通的列转化为行索引，行索引变为普通的列是经常出现的需求，本节则致力于讲清楚其转化的方式。

    @auther: sunzhenhang
    @date: 2017-08-18

In [2]:
!cd

E:\ML\实战\pandas实用教程


In [3]:
import numpy as np
import pandas as pd

In [4]:
data = [[1,2,3],[4,5,6]]
index = ['a','b']
columns = ['A','B','C']
df = pd.DataFrame( data = data, index = index, columns = columns)
df

,A,B,C
a,1,2,3
b,4,5,6


 ---
# 1. 普通列 转化为 行索引

#### `DataFrame.set_index(keys, drop=True, append=False, inplace=False)`
- keys：用于转化的列名称，列名或列名的列表，如果是列表则转化为多重索引；
- drop：True or False，是否保留原列；
- append：True or False，是否保留当前索引；
- inplace：True or False，是否原地修改。

## 1.1 DataFrame中的列
**注意：**下面的这种用法只能处理单层列索引情况。

In [5]:
df.set_index('A', drop = True, append = False) #单列转化为行索引，列名作为索引的名字

,B,C
A,,
1,2,3
4,5,6


In [6]:
df.set_index(['A','B'], drop = True, append = False) # 列名的列表

,,C
A,B,
1,2,3
4,5,6


In [7]:
df.set_index(['A','C'], drop = False, append = False) # drop = False, 保留原列

,,A,B,C
A,C,,,
1,3,1,2,3
4,6,4,5,6


In [8]:
df.set_index(['A','C'], drop = False, append = True) # append = True, 保留原索引

,,,A,B,C
,A,C,,,
a,1,3,1,2,3
b,4,6,4,5,6


--- 
## 1.2 任意列
- 普通列不是必须为DataFrame中的列，任何行数匹配的列表或**列表的元组**都可以；
- 此方法可以处理多层列索引情况，方法是先提取需要的列，构建成元组，再来替换索引。

In [31]:
li = ['c','d']
df.set_index(keys = [li], append = False)# 注意是[li]不是li，如果是单层，会当做列名处理
## append为False，实现了改索引功能

,A,B,C
c,1,2,3
d,4,5,6


In [43]:
#如果是多层索引情况，append为False，所有索性会被替换
df1 = df.set_index(keys = [li], append = True)
df1

,,A,B,C
a,c,1,2,3
b,d,4,5,6


In [51]:
df1.set_index(keys = [['e','f']],append = False)

,A,B,C
e,1,2,3
f,4,5,6


In [58]:
# 元祖的列表建立多层索引
list_tuple = [('e','f'),('g','h')]
df1.set_index(keys = [list_tuple],append = False)

,,A,B,C
e,f,1,2,3
g,h,4,5,6


---
# 2. 行索引 转化为 普通列

#### `DataFrame.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')`
- level：int 或者行索引的名字，可以为列表，默认包含所有行索引。
- drop：True or False，是否将行索引插入到列中，默认是插入；
- inplace：是否本地修改；
- col_level：如果列索引也是多重的，那么新插入的列设置哪一重索引；
- col_fill：如果有多重索引，除了col_level已经设置的

## 2.1 单层 列索引 情况

In [10]:
df1 = df.set_index(['A','C'], drop = True, append = True)
df1 

,,,B
,A,C,
a,1,3,2
b,4,6,5


In [11]:
# df1.reset_index(level = 'A', drop = False)
df1.reset_index(level = 1, drop = False) #单 行索引

,,A,B
,C,,
a,3,1,2
b,6,4,5


In [12]:
df1.reset_index(level = [1,2], drop = False) # 行索引的列表

,A,C,B
a,1,3,2
b,4,6,5


In [13]:
#如果所有行索引都恢复为列，那么将生成默认数值索引，如果行索引没有名字，那么生成默认的名字
df1.reset_index(level = [0,1,2], drop = False) 

,level_0,A,C,B
0,a,1,3,2
1,b,4,6,5


In [14]:
df1.index.set_names(names = 'ab', level = 0, inplace = True)
df1

,,,B
ab,A,C,
a,1,3,2
b,4,6,5


In [15]:
#行索引如果有名字，那么将其设置为列名
df1.reset_index(level = [0,1,2], drop = False) 

,ab,A,C,B
0,a,1,3,2
1,b,4,6,5


In [16]:
df1.reset_index(level = [1,2], drop = True) # drop = True, 直接丢掉选中的行索引，不插入

,B
ab,
a,2
b,5


---
## 2.2 多层 列索引 情况

In [17]:
df1 = df.copy()
df1.columns = pd.MultiIndex.from_tuples( [('one','A'),('one','B'),('two','C')])
df1

one    two
    A  B   C
a   1  2   3
b   4  5   6

In [18]:
#多重索引下，如果行索引没有名字，那么变为列之后在列索引的外层生成了默认索引：index。
df1.reset_index(level = 0)

index one    two
          A  B   C
0     a   1  2   3
1     b   4  5   6

In [27]:
#多重索引下，如果行索引有名字，那么变为列之后在列索引的外层索引名作为列名。
df1.index.set_names(['ab'], inplace = True)
df1

one    two
     A  B   C
ab           
a    1  2   3
b    4  5   6

In [28]:
df1.reset_index(level = 0)

ab one    two
       A  B   C
0  a   1  2   3
1  b   4  5   6